In [1]:
#Import all required libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import substring, length, col, expr
        



In [2]:
spark=SparkSession\
        .builder\
        .appName("midTermQuestion2")\
        .getOrCreate()
spark

In [3]:
sc=spark.sparkContext
sc

<SparkContext master=local[*] appName=midTermQuestion2>

In [5]:
# Read all files
# Keep updated records +
# Keep Old records which are not present in current month

### Settings ###
lastFile='2019-12'   # -- starting file --> last latest month
lastYear=int(lastFile.split('-')[0]) #-- Don't change
lastMonth=int(lastFile.split('-')[1]) #-- Don't change
totalFiles=9   # How many months to procees 
### Settings ###

dataCountBeforeFiltring=[]
dataCountAfterFiltring=[]
combinedData=[]
fileNames=[]

for i in range(totalFiles):
    # File Storage path
    path="C:\\Users\dhava\Downloads\Dell Repair\HW\PBDA\Project\Dataset\Airbnb\\2019\listings\small\listings-"+str(lastYear)+"-"+str(lastMonth-i).zfill(2)+".csv"
    fileNames.append(str(lastYear)+"-"+str(lastMonth-i).zfill(2))
    clearPath=path.replace('\\','/')
    dfSingle=spark.read \
            .format("csv") \
            .option("header", "true") \
            .option("inferSchema","true") \
            .load(clearPath)
    
    # Saving data for visilization
    dataCountBeforeFiltring.append(dfSingle.count())
    # Remove data if pricing column has null values
    dfSingle=dfSingle.filter(dfSingle.price.isNotNull())  
    # Convert the pricing column to float
    dfSingle=dfSingle.withColumn("price",expr("substring(price, 2, length(price))").cast("float"))
    dfSingle=dfSingle.filter(dfSingle.price.isNotNull())
    # Saving data for visilization
    dataCountAfterFiltring.append(dfSingle.count())
    if(i==0):
        dfCombined=dfSingle   # Select the 1st file --> which is latest 12th Month
    else:
        # Now work with other months --> going by 11, 10, 9, and so on...
        # According to the data analysis, 
        # Every month, around 1000 new listings are been added and 3000 listings get delisted in the current month
        # For future use, We are combining the both months data using join operations and are maintaing dataset with listings from both months
        
        # Thus, Dataset has 
        # Listings which were present last month but right now got delisted 
        # + listings which are present in current month and were also present last month
        # + listings which are present only in this month and were not present last month
        dfCombined.createOrReplaceTempView("tCombined")
        dfSingle.createOrReplaceTempView("tSingle")
        newWithOldUpdated=spark.sql('''select tc.* from tCombined as tc left join tSingle as ts on tc.id=ts.id''')
        oldWhichIsNotPresent=spark.sql('''select ts.* from tSingle as ts left join tCombined as tc on tc.id=ts.id where tc.id is null''')
        dfCombined=newWithOldUpdated.union(oldWhichIsNotPresent)
    combinedData.append(dfCombined.count())
    
    print("Done Combining:",clearPath)
    

Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-12.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-11.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-10.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-09.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-08.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-07.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-06.csv
Done Combining: C:/Users/dhava/Downloads/Dell Repair/HW/PBDA/Project/Dataset/Airbnb/2019/listings/small/listings-2019-05.csv


In [7]:
#Making Visualization

import numpy as np
import matplotlib.pyplot as plt

# Ref: https://pythonspot.com/matplotlib-bar-chart/
# data to plot
n_groups = totalFiles
largeValues =dataCountBeforeFiltring
smallValues =dataCountAfterFiltring


from matplotlib.pyplot import figure
plt.figure(num=None, figsize=(18, 6), dpi=800, facecolor='w', edgecolor='k')

# create plot
index = np.arange(n_groups)
bar_width = 0.35
opacity = 0.8

rects1=plt.bar(index, largeValues, bar_width,alpha=opacity,color='orange',label='Before Filtering')
label=largeValues
for i in range(len(largeValues)):
    plt.text(x = index[i]-0.2, y = largeValues[i]+0.3, s = label[i], size = 8)

rects2 = plt.bar(index + bar_width, smallValues, bar_width,alpha=opacity,color='lightblue',label='After Filtering')
label=smallValues
for i in range(len(smallValues)):
    plt.text(x =index[i]+0.2, y = smallValues[i]+0.3, s = label[i], size = 8)

plt.xlabel('Timeline')
plt.ylabel('Data Size')
plt.title('Data size by Months')
plt.xticks(index + bar_width, fileNames)
plt.rcParams["legend.loc"] = 'lower right' 
plt.legend()
plt.show()

In [14]:
# data to plot
n_groups = totalFiles
largeValues =combinedData
plt.figure(num=None, figsize=(18, 6), dpi=800, facecolor='w', edgecolor='k')
rects1=plt.bar(index,largeValues,alpha=opacity,color='lightblue')
label=largeValues
for i in range(len(largeValues)):
    plt.text(x = index[i]-0.1, y = largeValues[i], s = label[i], size = 8)
plt.xlabel('Timeline')
plt.ylabel('Combined Data Frame Size')
plt.title('Data size of combined data frame after past months files have been successfully combined')
plt.xticks(index, fileNames)
plt.rcParams["legend.loc"] = 'lower right' 
plt.show()

In [ ]:
dfCombined.count()

In [ ]:
# Save file --> given to Yash for further ETL
dfCombined.toPandas().to_csv("combinedData.csv")

In [ ]:
## Code to be added by Yash

In [ ]:
print(smallValues)
print(largeValues)

In [ ]:
## Code to be added by Vishnu


# Sample data for Training 
## If needed for ML Traingin

sampleData=dfWithRatings.sample(withReplacement=True, fraction=0.01, seed=42)
sampleData.count()

data=sampleData
train, test = data.randomSplit([0.9, 0.1], seed=12345)


